### Loading necessary libraries:

In [17]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

### Reading the data from the files:

In [20]:
exp_invoices_path = "data/expired_invoices.txt"
with open(exp_invoices_path, 'r') as file:
    expired_invoices = list(map(int, file.read().strip().split(', ')))

invoices_path = "data/invoices_new.pkl"
with open(invoices_path, 'rb') as file:
    invoices = pickle.load(file)

invoices[0]

{'id': '365371O',
 'created_on': '2019-05-27',
 'items': [{'item': {'id': 142446,
    'name': 'ii_142446',
    'unit_price': 182,
    'type': 1},
   'quantity': 6},
  {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1},
   'quantity': 1}]}

In [25]:
import pandas as pd
import pickle
from datetime import datetime

class DataExtractor:
    def __init__(self, invoices_path, expired_invoices_path):
        self.invoices_path = invoices_path
        self.expired_invoices_path = expired_invoices_path
        self.invoices = None
        self.expired_invoices = None
    
    def load_data(self):
        # Load expired invoice IDs
        with open(self.expired_invoices_path, 'r') as file:
            self.expired_invoices = list(map(int, file.read().strip().split(', ')))
        
        # Load invoices data
        with open(self.invoices_path, 'rb') as file:
            self.invoices = pickle.load(file)
    
    def clean_invoice_id(self, invoice_id):
        # Remove non-numeric characters and convert to integer
        return int(''.join(filter(str.isdigit, str(invoice_id))))
    
    def safe_convert_to_int(self, value, default=0):
        # Convert a value to integer, returning a default value if conversion fails
        try:
            return int(value)
        except (ValueError, TypeError):
            return default
    
    def safe_map_type(self, type_value, type_mapping, default='Other'):
        # Map type value to its string representation, return default if not found
        return type_mapping.get(type_value, default)
    
    def transform_data(self):
        invoice_list = []
        type_mapping = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
        
        for invoice in self.invoices:
            # Clean and convert invoice_id to integer
            invoice_id = self.clean_invoice_id(invoice['id'])
            created_on = pd.to_datetime(invoice['created_on'])
            
            # Check if 'items' key exists in the invoice
            if 'items' not in invoice:
                continue
            
            # Calculate the total value of the invoice
            invoice_total = sum(self.safe_convert_to_int(item['item']['unit_price']) * self.safe_convert_to_int(item['quantity']) for item in invoice['items'])
            
            # Determine if the invoice is expired
            is_expired = invoice_id in self.expired_invoices
            
            for item in invoice['items']:
                invoiceitem_id = item['item']['id']
                invoiceitem_name = item['item']['name']
                type_str = self.safe_map_type(item['item']['type'], type_mapping)
                unit_price = self.safe_convert_to_int(item['item']['unit_price'])
                quantity = self.safe_convert_to_int(item['quantity'])
                total_price = unit_price * quantity
                percentage_in_invoice = total_price / invoice_total if invoice_total != 0 else 0
                
                # Append the processed item data to the list
                invoice_list.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': invoiceitem_id,
                    'invoiceitem_name': invoiceitem_name,
                    'type': type_str,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })
        
        # Convert the list to a DataFrame
        df = pd.DataFrame(invoice_list)
        
        # Ensure the correct data types for each column
        df = df.astype({
            'invoice_id': 'int64',
            'created_on': 'datetime64[ns]',
            'invoiceitem_id': 'int64',
            'invoiceitem_name': 'str',
            'type': 'str',
            'unit_price': 'int64',
            'total_price': 'int64',
            'percentage_in_invoice': 'float64',
            'is_expired': 'bool'
        })
        
        # Sort the DataFrame by invoice_id and invoiceitem_id
        df = df.sort_values(by=['invoice_id', 'invoiceitem_id']).reset_index(drop=True)
        return df

# Example usage
invoices_path = "data/invoices_new.pkl"
expired_invoices_path = "data/expired_invoices.txt"
extractor = DataExtractor(invoices_path, expired_invoices_path)
extractor.load_data()
df = extractor.transform_data()

import ace_tools as tools; tools.display_dataframe_to_user(name="Transformed Invoice Data", dataframe=df)

# Save the transformed dataframe to a CSV file
df.to_csv('data/transformed_invoices.csv', index=False)


DateParseError: day is out of range for month: 2019-02-30, at position 0